In [1]:
import json
import pandas as pd
import pyperclip

## CLI get JSON data

In [2]:
def get_json_list_functions(region, output_file='Downloads/aws_lambda_functions.json', copy=True):
    _cli = f'aws lambda list-functions --region {region} --output json > {output_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

In [3]:
def get_json_list_layers(region, output_file='Downloads/aws_lambda_layers.json', copy=True):
    _cli = f'aws lambda list-layers --region {region} --output json > {output_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

In [4]:
def get_json_get_policy(function_name, output_file='Downloads/aws_lambda_policy.json', copy=True):
    _cli = f'aws lambda get-policy --function-name {function_name} --output text > {output_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

In [5]:
get_json_list_functions('ap-east-1')

'aws lambda list-functions --region ap-east-1 --output json > Downloads/aws_lambda_functions.json'

## Load JSON data

In [6]:
def frame_policy(json_file):
    with open(json_file) as f:
        _full_text = f.read()
        _json_text = text.split('\t')[0]
        _data = json.loads(_json_text)['Statement']
    
    for i in _data:
        i['LogGroup'] = i['Condition']['ArnLike']['AWS:SourceArn']
        i['FunctionName'] = i['Condition']['ArnLike']['AWS:SourceArn'].split('/')[-1].split(':')[0]
    _df = pd.DataFrame(_data)
    return _df

In [7]:
def frame_functions(json_file):
    with open(json_file) as f:
        _data = json.load(f)['Functions']
    for function in _data:
        try:
            function['Layers'] = " ".join([layer['Arn'] for layer in function['Layers']])
        except:
            pass
    _df = pd.DataFrame(_data)
    return _df

In [8]:
def dict_layer_latest_arn(json_file):
    with open(json_file) as f:
        _data = json.load(f)['Layers']
        
    _layer_latest_arn = {}
    for layer in _data:
        _name = layer['LayerName']
        _arn = layer['LatestMatchingVersion']['LayerVersionArn']
        _layer_latest_arn[_name] = _arn
        
    return _layer_latest_arn

## CLI update function configuration

In [9]:
def cli_update_memory_size(function_name, memory_size=128, log_file='Downloads/aws_log.txt', copy=True):
    _cli = f'aws lambda update-function-configuration --function-name {function_name} --memory-size {memory_size} >> {log_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

In [10]:
def cli_update_ephemeral_storage(function_name, size=512, log_file='Downloads/aws_log.txt', copy=True):
    _cli = f'aws lambda update-function-configuration --function-name {function_name} --ephemeral-storage Size={size} >> {log_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

In [11]:
def cli_update_layers(function_name, layers_arn, log_file='Downloads/aws_log.txt', copy=True):
    _cli = f'aws lambda update-function-configuration --function-name {function_name} --layers {layers_arn} >> {log_file}'
    if copy:
        pyperclip.copy(_cli)
    return _cli

## CLI bulk update functions

### Bulk replace layers

In [12]:
def replace_layer(layers, old_layer_name, new_layer_arn):
    _layers = layers.split(' ')
    _new_layers = ' '.join([new_layer_arn if old_layer_name == layer.split(':')[-2] else layer for layer in _layers])
    return _new_layers

In [13]:
def bulk_replace_layers(functions_json_file, replacements_dict, copy=True):
    _df = frame_functions(functions_json_file)
    _df = _df[~pd.isna(_df.Layers)]
    
    for key in replacements_dict:
        old_layer_name = key
        new_layer_arn = replacements[key]
        _df.Layers = _df.apply(lambda x: replace_layer(x.Layers, old_layer_name, new_layer_arn), axis=1)
    
    _df['cli'] = _df.apply(lambda x: cli_update_layers(x.FunctionName, x.Layers), axis=1)
    
    _cli = ';'.join(_df['cli'].values.tolist())
    
    if copy:
        pyperclip.copy(_cli)

    return _cli

In [14]:
layer_latest_arn = dict_layer_latest_arn('aws_lambda_layers.json')
layer_latest_arn

{'aws-data-wrangler': 'arn:aws:lambda:ap-east-1:238911357348:layer:aws-data-wrangler:1',
 'gspread-plus': 'arn:aws:lambda:ap-east-1:238911357348:layer:gspread-plus:2',
 'httplib2': 'arn:aws:lambda:ap-east-1:238911357348:layer:httplib2:1',
 'kam-credentials': 'arn:aws:lambda:ap-east-1:238911357348:layer:kam-credentials:4',
 'pandas_numpy_cp39': 'arn:aws:lambda:ap-east-1:238911357348:layer:pandas_numpy_cp39:1',
 'pydrive': 'arn:aws:lambda:ap-east-1:238911357348:layer:pydrive:1',
 'tenacity': 'arn:aws:lambda:ap-east-1:238911357348:layer:tenacity:1'}

In [15]:
replacements = {
    'kam-credentials': layer_latest_arn['kam-credentials'],
    'aws-data-wrangler': layer_latest_arn['pandas_numpy_cp39'],
    'gspread': layer_latest_arn['gspread-plus'],
    'gspread-plus': layer_latest_arn['gspread-plus']
}

In [16]:
cli = bulk_replace_layers('aws_lambda_functions.json', replacements)

### Bulk update sample

In [17]:
def bulk_update_functions(function_names_list, cli_function, value=None, values_list=None, copy=True):
    _cli_list = []
    if (value != None) and (values_list == None):
        for function_name in function_names_list:
            _cli_list.append(cli_function(function_name, value))
    elif (value == None) and (values_list != None):
        for function_name, value in zip(function_names_list, values_list):
            _cli_list.append(cli_function(function_name, value))
    else:
        raise ValueError('Please input value or value_list only')
            
    _cli = ';'.join(_cli_list)
    
    if copy:
        pyperclip.copy(_cli)

    return _cli

In [18]:
bulk_update_functions(['abc', 'def'], cli_update_memory_size, values_list=[10, 5])

'aws lambda update-function-configuration --function-name abc --memory-size 10 >> Downloads/aws_log.txt;aws lambda update-function-configuration --function-name def --memory-size 5 >> Downloads/aws_log.txt'